# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298581015146                   -0.85    4.8
  2   -8.300213484834       -2.79       -1.25    1.0
  3   -8.300448856085       -3.63       -1.87    4.0
  4   -8.300462086650       -4.88       -2.72    1.6
  5   -8.300464024136       -5.71       -3.09    4.9
  6   -8.300464314923       -6.54       -3.24    5.4
  7   -8.300464473640       -6.80       -3.38    2.0
  8   -8.300464561799       -7.05       -3.52    2.1
  9   -8.300464630258       -7.16       -3.76    1.2
 10   -8.300464636714       -8.19       -3.89    5.9
 11   -8.300464643604       -8.16       -4.32    1.4


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64194998994                   -0.70    5.6
  2   -16.64595948030       -2.40       -1.14    1.0
  3   -16.67921180786       -1.48       -1.86    4.1
  4   -16.67927362173       -4.21       -2.66    3.2
  5   -16.67928570305       -4.92       -3.01    5.8
  6   -16.67928620965       -6.30       -3.33    1.5
  7   -16.67928622042       -7.97       -3.84    1.0
  8   -16.67928622412       -8.43       -4.58    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.30933102808                   -0.56    6.6
  2   -33.33445783943       -1.60       -0.99    1.4
  3   -33.33663105642       -2.66       -1.69    4.1
  4   -33.33692309040       -3.53       -2.60    2.8
  5   -33.33694186052       -4.73       -3.11   12.4
  6   -33.33694386681       -5.70       -3.92    5.1
  7   -33.33694391215       -7.34       -4.39    9.5


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298671951488                   -0.85    4.0
  2   -8.300294069714       -2.79       -1.59    1.5
  3   -8.300456526402       -3.79       -2.61    5.9
  4   -8.300408173587   +   -4.32       -2.38   14.5
  5   -8.300464538923       -4.25       -3.73    3.2
  6   -8.300464609907       -7.15       -4.05    3.4


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.31503790570                   -0.56    6.2
  2   -33.30070356422   +   -1.84       -1.26    1.0
  3   -10.66356913597   +    1.35       -0.39    7.9
  4   -33.32316469079        1.36       -1.69    8.4
  5   -32.95307904861   +   -0.43       -1.19    4.5
  6   -31.47559179359   +    0.17       -0.93    4.9
  7   -33.32049065996        0.27       -1.87    6.0
  8   -33.30439061070   +   -1.79       -1.59    3.1
  9   -33.33600774763       -1.50       -2.49    2.6
 10   -33.33679643007       -3.10       -2.81    3.0
 11   -33.33689974763       -3.99       -3.08    3.4
 12   -33.33692144162       -4.66       -3.33    2.6
 13   -33.33694098924       -4.71       -3.67    3.1
 14   -33.33694288681       -5.72       -3.88    2.6
 15   -33.33693000089   +   -4.89       -3.49    4.1
 16   -33.33694378995       -4.86       -4.41    4.4


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.